# Figure S9 data: human inulin association maps
This script generates xml files for the networks showing changes in associations for Bifidobacterium and Agathobacter rectalis in figure S9 of the paper. The xml files can be viewed in cytoscape and combined in adobe illustrator to generate figure S9.

### Before you start
This notebook assumes the analysis on the human inulin dataset have been run and results are located in `MCSPACE_paper/results/analysis/human_inulin`. Refer to the README in `scripts/analysis` for the analysis pipeline and more details.

In [1]:
from pathlib import Path
import pandas as pd
from mcspace.utils import pickle_load
import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np

from mcspace.visualization import render_assemblages, render_assemblage_proportions, export_association_networks_to_cytoscape
import matplotlib.colors as mcolors
import mcspace.vis_tools as vis
import matplotlib.patches as mpatches

In [2]:
%matplotlib inline 

In [3]:
# change font to arial
mpl.rcParams['font.sans-serif'] = "Arial"
mpl.rcParams['font.family'] = "sans-serif"
plt.rcParams['svg.fonttype'] = 'none'

# Paths

In [4]:
basepath = Path("./")
rootpath = Path("../../")
datapath = rootpath / "MCSPACE_paper" / "datasets"

outpath = basepath / "supplemental_figures"
outpath.mkdir(exist_ok=True, parents=True)

In [5]:
resultpath = rootpath / "MCSPACE_paper" / "results" / "analysis" / "human_inulin"

In [7]:
results = pickle_load(resultpath / "results.pkl")

# Fix taxonomy (add genus to species names...)

In [8]:
results.keys()

dict_keys(['assemblages', 'assemblage_proportions', 'perturbation_bayes_factors', 'relative_abundances'])

In [9]:
theta = results["assemblages"]

In [10]:
print(theta.shape)

(50, 42)


In [11]:
theta.head()

,,,,,,,,A1,A2,A3,A4,A5,A6,A7,A8,A9,A10,...,A33,A34,A35,A36,A37,A38,A39,A40,A41,A42
Otu,Domain,Phylum,Order,Class,Family,Genus,Species,,,,,,,,,,,,,,,,,,,,,
Otu15,Bacteria,Firmicutes,Acidaminococcales,Negativicutes,Acidaminococcaceae,Acidaminococcaceae,na,5.402044e-10,4.566318e-10,7.087878e-10,6.761923e-10,1.011743e-02,4.154032e-03,6.815317e-10,6.337994e-04,9.157046e-10,1.710469e-08,...,2.753280e-09,9.905521e-10,0.021291,1.765768e-09,6.548908e-08,1.364419e-09,1.392650e-09,8.368565e-04,6.111267e-03,5.314949e-04
Otu81,Bacteria,Firmicutes,Clostridia UCG-014,Clostridia,na,na,na,5.993243e-10,1.099598e-09,1.125636e-09,7.870884e-10,7.910649e-10,8.462128e-10,3.655898e-09,3.735886e-05,1.654418e-03,1.180011e-03,...,3.719087e-04,4.524394e-04,0.012844,2.875757e-09,6.175879e-08,1.043484e-03,1.019878e-02,1.219343e-09,7.642739e-05,1.901888e-09
Otu109,Bacteria,Firmicutes,Lachnospirales,Clostridia,Lachnospiraceae,na,na,1.223170e-09,1.269643e-09,4.571187e-05,9.508778e-10,2.440368e-09,6.747477e-03,5.486141e-10,2.234653e-09,7.103887e-10,1.271684e-09,...,1.121878e-03,6.744422e-10,0.002810,1.796872e-09,1.280230e-07,8.011021e-10,3.995764e-02,2.639358e-10,2.562632e-03,1.772351e-02
Otu52,Bacteria,Firmicutes,Lachnospirales,Clostridia,Lachnospiraceae,Lachnospiraceae,UCG-001,1.028309e-03,7.469828e-10,3.630410e-06,9.249437e-10,1.098402e-09,1.015128e-02,1.650840e-04,1.012947e-03,1.530720e-09,1.035948e-04,...,4.415186e-04,2.446390e-09,0.003712,2.437058e-03,1.112596e-03,3.667009e-03,1.304979e-09,2.938695e-09,9.657782e-10,3.487537e-04
Otu75,Bacteria,Actinobacteriota,Coriobacteriales,Coriobacteriia,Eggerthellaceae,na,na,7.534840e-04,2.488229e-05,8.029230e-10,9.289624e-10,1.575106e-09,7.921071e-10,7.187091e-10,6.652289e-10,1.214494e-04,9.285728e-04,...,8.868053e-03,4.765187e-04,0.002964,5.740490e-09,1.374571e-07,8.202014e-10,3.937318e-03,3.687582e-03,2.894270e-04,6.012646e-04


In [12]:
names=['Otu', 'Domain', 'Phylum', 'Order', 'Class', 'Family', 'Genus', 'Species']

new_index = [
    (otu, domain, phylum, order, class_, family, genus, f"{genus} {species}" if species != "na" else "na")
    for otu, domain, phylum, order, class_, family, genus, species in theta.index
]

In [13]:
theta.index = pd.MultiIndex.from_tuples(new_index, names=names)

In [14]:
theta.head()

,,,,,,,,A1,A2,A3,A4,A5,A6,A7,A8,A9,A10,...,A33,A34,A35,A36,A37,A38,A39,A40,A41,A42
Otu,Domain,Phylum,Order,Class,Family,Genus,Species,,,,,,,,,,,,,,,,,,,,,
Otu15,Bacteria,Firmicutes,Acidaminococcales,Negativicutes,Acidaminococcaceae,Acidaminococcaceae,na,5.402044e-10,4.566318e-10,7.087878e-10,6.761923e-10,1.011743e-02,4.154032e-03,6.815317e-10,6.337994e-04,9.157046e-10,1.710469e-08,...,2.753280e-09,9.905521e-10,0.021291,1.765768e-09,6.548908e-08,1.364419e-09,1.392650e-09,8.368565e-04,6.111267e-03,5.314949e-04
Otu81,Bacteria,Firmicutes,Clostridia UCG-014,Clostridia,na,na,na,5.993243e-10,1.099598e-09,1.125636e-09,7.870884e-10,7.910649e-10,8.462128e-10,3.655898e-09,3.735886e-05,1.654418e-03,1.180011e-03,...,3.719087e-04,4.524394e-04,0.012844,2.875757e-09,6.175879e-08,1.043484e-03,1.019878e-02,1.219343e-09,7.642739e-05,1.901888e-09
Otu109,Bacteria,Firmicutes,Lachnospirales,Clostridia,Lachnospiraceae,na,na,1.223170e-09,1.269643e-09,4.571187e-05,9.508778e-10,2.440368e-09,6.747477e-03,5.486141e-10,2.234653e-09,7.103887e-10,1.271684e-09,...,1.121878e-03,6.744422e-10,0.002810,1.796872e-09,1.280230e-07,8.011021e-10,3.995764e-02,2.639358e-10,2.562632e-03,1.772351e-02
Otu52,Bacteria,Firmicutes,Lachnospirales,Clostridia,Lachnospiraceae,Lachnospiraceae,Lachnospiraceae UCG-001,1.028309e-03,7.469828e-10,3.630410e-06,9.249437e-10,1.098402e-09,1.015128e-02,1.650840e-04,1.012947e-03,1.530720e-09,1.035948e-04,...,4.415186e-04,2.446390e-09,0.003712,2.437058e-03,1.112596e-03,3.667009e-03,1.304979e-09,2.938695e-09,9.657782e-10,3.487537e-04
Otu75,Bacteria,Actinobacteriota,Coriobacteriales,Coriobacteriia,Eggerthellaceae,na,na,7.534840e-04,2.488229e-05,8.029230e-10,9.289624e-10,1.575106e-09,7.921071e-10,7.187091e-10,6.652289e-10,1.214494e-04,9.285728e-04,...,8.868053e-03,4.765187e-04,0.002964,5.740490e-09,1.374571e-07,8.202014e-10,3.937318e-03,3.687582e-03,2.894270e-04,6.012646e-04


In [15]:
results["assemblages"] = theta

In [16]:
bf = results['perturbation_bayes_factors']

In [17]:
np.amax(bf)

3.0916824

## Aggregate time points by diet phase and output association network data

In [27]:
from mcspace import utils as ut

In [21]:
ra = results['relative_abundances']

In [18]:
diet_times = {'Pre-Inulin': [1,2,4],
'Inulin': [7,8,10],
'Post-Inulin': [11]}

In [19]:
def average_diet_times(df, diet_times):
    df2 = df.copy()
    diets_keep = list(diet_times.keys())
    for k in diets_keep:
        df2[k] = df.loc[:,diet_times[k]].mean(axis=1)
    df2 = df2[diets_keep]
    return df2

In [23]:
radiets = average_diet_times(ra, diet_times)

In [24]:
thetadf = results['assemblages']
betadf = results['assemblage_proportions']

In [28]:
avebeta = ut.get_subj_averaged_assemblage_proportions(betadf)

In [25]:
taxlevels = ['Otu', 'Domain', 'Phylum', 'Class', 'Order', 'Family', 'Genus', 'Species']
taxonomy = thetadf.reset_index()[taxlevels].copy()
taxonomy = taxonomy.set_index("Otu")

In [26]:
otu_plot = ["Otu1", "Otu6"]

In [30]:
for oidx in otu_plot:
    otu_name = ut.get_lowest_level_name(oidx, taxonomy)

    # get edges and node weights
    alpha = ut.get_assoc_scores(thetadf, avebeta, oidx)
    alphasub = average_diet_times(alpha, diet_times)
    ew = ut.filter_assoc_scores(alphasub, radiets, oidx, ra_threshold=0.01, edge_threshold=0.01)
    nw = radiets.loc[ew.index,:]

    # update labels for taxa 
    nw3 = ut.update_names(nw, taxonomy)
    ew3 = ut.update_names(ew, taxonomy)
    
    nw3.to_csv(outpath / f"node_data_{oidx}.csv")
    ew3.to_csv(outpath / f"edge_data_{oidx}.csv")

    # output to file
    ut.output_association_network_to_graphML(oidx, nw3, ew3, taxonomy, outpath / f'{otu_name}.xml')